# Model Output Notebook

<img style="float:center;" src="https://arcticexpansion.vse.gmu.edu/sites/arcticexpansion.vsnet.gmu.edu/files/images/header5d2.png" width=600px>

### ADCIRC-SWAN Output


### Initialize Libraries

In [1]:
import warnings;warnings.filterwarnings("ignore")
import netCDF4 as nc4;        import pandas as pd
import requests;              import json;
import matplotlib as mpl;     import matplotlib.pyplot as plt
import matplotlib.tri as tri; import pathlib as pl

import numpy as np;           import xarray as xr
import skill_metrics as sm;   import geopandas as gpd

import wse
import dask.array as da
from dask.diagnostics import ProgressBar
from shapely import Polygon,Point,MultiPoint,LineString,MultiLineString;import shapely.vectorized
from sklearn.neighbors import BallTree
from scipy.stats import linregress
from scipy.signal import find_peaks
from scipy import sparse
from scipy.sparse.csgraph import connected_components
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from multiprocessing import Pool

In [8]:
def preprocess(ds, year, keep_idx=None):
    """
    For one year’s dataset:
    - Use the file’s own time values (decoded by xarray).
    - Keep only timestamps within the calendar year (drops spin‑up & overrun).
    """
    # Convert the decoded time coordinate to pandas DatetimeIndex
    time = pd.to_datetime(ds.time.values)
    #year = time[0].year

    # Build the calendar‐year mask
    start = pd.to_datetime(f"{year}-01-01T00:00:00")
    end   = pd.to_datetime(f"{year}-12-31T23:00:00")
    mask  = (time >= start) & (time <= end)
    ds.isel(time=mask)
    if keep_idx is not None:
        ds = ds.isel(node=keep_idx)
    
    return ds
def open_zeta_dataset(root_path, years, chunks={"time":1000, "node":20000}):
    """
    Open each year’s CF‑ified file with mask_and_scale=False (no NaNs),
    preprocess it, then concat along time.
    """
    ds_list = []
    for year in years:
        path = pl.Path(root_path) / str(year) / "fort.63.cf.nc"
        # disable automatic masking of the fill_value
        ds = xr.open_dataset(path,
                             engine="netcdf4",
                             chunks=chunks,
                             mask_and_scale=False)
        ds = preprocess(ds, year)
        ds_list.append(ds)
    return xr.concat(ds_list, dim="time")
def build_adjacency_from_sample(path):
    """
    Build sparse node‐to‐node adjacency from one sample CF file.
    """
    ds0 = xr.open_dataset(path, engine="netcdf4")
    elems = ds0["face_node_connectivity"].values - 1  # shape (nele, 3)
    ds0.close()

    n = elems.max() + 1
    row = np.hstack([elems[:,0], elems[:,1],
                     elems[:,1], elems[:,2],
                     elems[:,2], elems[:,0]])
    col = np.hstack([elems[:,1], elems[:,0],
                     elems[:,2], elems[:,1],
                     elems[:,0], elems[:,2]])
    data = np.ones_like(row, dtype=int)
    return sparse.coo_matrix((data, (row, col)), shape=(n, n)).tocsr()

def compute_3h_max(ds):
    """
    Compute rolling 3‑hour max on ds.zeta, keeping only the first
    two “incomplete” hours as NaN and then slicing them off.
    """
    z = ds.zeta
    # perform a 3‑step window max, requiring all 3 points
    surge3 = z.rolling(time=3, center=False, min_periods=3).max()
    # drop the first two timestamps, which will be NaN
    return surge3.isel(time=slice(2, None))
def extract_thresholds(surge3):
    # Compute in‑place on Dask arrays; returns two DataArrays
    p95_da  = surge3.quantile(0.95,  dim="time", skipna=True)
    p999_da = surge3.quantile(0.999, dim="time", skipna=True)
    # Now pull the resulting 1D arrays into memory
    th1 = p95_da.data.compute()
    th2 = p999_da.data.compute()
    return th1, th2

def detect_node_peaks(surge3, th2, chunk_size=1000):
    """
    Stream through nodes in chunks, loading only chunk_size columns at once.
    Returns dict: global_node_index -> array of peak time‐indices.
    """
    from scipy.signal import find_peaks

    n_time = surge3.sizes["time"]
    n_node = surge3.sizes["node"]
    peaks = {}

    for i in range(0, n_node, chunk_size):
        j = min(n_node, i + chunk_size)
        # load a small block: shape = (time, j-i)
        block = surge3.isel(node=slice(i, j)).values
        th2_block = th2[i:j]

        for local_idx in range(block.shape[1]):
            idxs, _ = find_peaks(
                block[:, local_idx],
                height=th2_block[local_idx],
                distance=24
            )
            if idxs.size:
                peaks[i + local_idx] = idxs

    return peaks


def find_spatial_footprints(surge3, th1, th2, adjacency):
    """
    Iterate over time steps, loading one time slice at a time.
    Returns dict: time_index -> label array for all nodes.
    """
    from scipy.sparse.csgraph import connected_components

    n_node = surge3.sizes["node"]
    footprints = {}

    for t in range(surge3.sizes["time"]):
        # load only one timestep: shape = (node,)
        z_t = surge3.isel(time=t).values
        # quick check if any node exceeds the high threshold
        if not (z_t >= th2).any():
            continue

        # mask at the lower threshold and cluster
        mask1 = z_t >= th1
        nodes = np.nonzero(mask1)[0]
        sub_adj = adjacency[nodes][:, nodes]
        labels, _ = connected_components(sub_adj, directed=False)

        full_labels = -1 * np.ones(n_node, dtype=int)
        full_labels[nodes] = labels
        footprints[t] = full_labels

    return footprints

def cluster_events(feature_matrix, n_clusters=8):
    """PCA + k‑means clustering of event features."""
    scores = PCA(n_components=6).fit_transform(feature_matrix)
    return KMeans(n_clusters=n_clusters, random_state=0).fit_predict(scores)


In [3]:
root = '/scratch/tmiesse/project/data4spatial'

In [4]:
YEARS = range(2023, 2025)
MAX_DEPTH = 20.0

sample_path = f"{root}/2023/fort.63.cf.nc"
adj = build_adjacency_from_sample(sample_path)
ds = open_zeta_dataset(root, YEARS)
ds     = ds.chunk({"time":1000, "node":20000})
print("Time span:", ds.time.values[0], "to", ds.time.values[-1])
print("Working on full mesh with", ds.sizes["node"], "nodes")

Time span: 2022-12-15T00:00:00.000000000 to 2024-12-29T23:00:00.000000000
Working on full mesh with 861878 nodes


In [5]:
# 3. Compute rolling 3h max
surge3 = compute_3h_max(ds)

In [6]:
# 4. Extract thresholds
th1, th2 = extract_thresholds(surge3)

In [ ]:
# 5. Detect peaks & footprints
peaks = detect_node_peaks(surge3, th2)
fps   = find_spatial_footprints(surge3, th1, th2, adj)
print(f"Found peaks on {len(peaks)} nodes, {len(fps)} time slices with footprints.")